<a href="https://colab.research.google.com/github/JacobDowns/CSCI-491-591/blob/main/linear_algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Algebra on GPUs

* Wow, this sounds super boring, why would anyone care?
* The solution of linear algebra problems arises frequently in scientific computing, data science, machine learning, and many other fields
* Linear algebra is a cornerstone of modern computing, so its an important topic!
* Problems that emerge frequently include
    * Factorizing matrices
    * Matrix vector multiplication
    * Matrix times matrix multiplication
    * Computing eigenvalues and eigenvectors
    * Solving linear systems
    * Sparse linear algebra
    * Among others
* A great deal of effort has gone into implementing efficient solutions to linear algebra problems over the years
* But not all algorithms that were developed for CPUs can be ported directly to GPUs while maintaining performance

## Example
* Let's say we want to solve $Ax = b$ for some big matrix $A$
* One pretty common way of doing this is by using some efficient algorithm to compute the LU decomposition (or a similar decomposition like a Cholesky decomposition)
$$
A = LU
$$
where $L$ is a lower tirangular matrix and $U$ is an upper triangular matrix.
* This approach is especially useful when the factorization only needs to be computed once and can be reused for multiple tasks
* The problem is now
$$
A = LUx = b
$$
which we can solve in two passes
* First, we solve
$$
Ly = b
$$
for $y$ and subsequently
$$
Ux = y
$$
for $x$, which by simple substitution satisfies the original problem $Ax=b$

* Conveniently, $Ly=b$ is easy to solve because $L$ is lower triangular.
* It can be solved using **forward substitution**
* Similarly, $Ux=y$ is easy to solve because $U$ is upper triangular
* It can be solved with **backward substitution**

## The Problem
* Let's assume for a minute that a CPU and GPU are equally good at computing the LU decomposition
> Indeed, there are some efficient algorithms for computing matrix factorizations on GPUs
* With that assumption, how good is a GPU at solving $Ly=b$ using forward substitution or $Ux=y$ with backward substitution?
* It's far from an ideal task for a GPU because its entirely **sequential**
* Solving for the next element depends on the solution for the previous element


$$
\textbf{Forward substitution:} \quad
L y = b, \quad
L =
\begin{bmatrix}
\ell_{11} & 0         & 0         & \cdots & 0 \\
\ell_{21} & \ell_{22} & 0         & \cdots & 0 \\
\ell_{31} & \ell_{32} & \ell_{33} & \cdots & 0 \\
\vdots    & \vdots    & \vdots    & \ddots & \vdots \\
\ell_{n1} & \ell_{n2} & \ell_{n3} & \cdots & \ell_{nn}
\end{bmatrix}
$$

$$
\Rightarrow \quad
\begin{aligned}
y_1 &= \frac{b_1}{\ell_{11}}, \\
y_2 &= \frac{b_2 - \ell_{21} y_1}{\ell_{22}}, \\
y_3 &= \frac{b_3 - \ell_{31} y_1 - \ell_{32} y_2}{\ell_{33}}, \\
&\ \vdots \\
y_n &= \frac{b_n - \sum_{j=1}^{n-1} \ell_{nj} y_j}{\ell_{nn}}.
\end{aligned}
$$

$$
\textbf{Backward substitution:} \quad
U x = y, \quad
U =
\begin{bmatrix}
u_{11} & u_{12} & u_{13} & \cdots & u_{1n} \\
0      & u_{22} & u_{23} & \cdots & u_{2n} \\
0      & 0      & u_{33} & \cdots & u_{3n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0      & 0      & 0      & \cdots & u_{nn}
\end{bmatrix}
$$\

$$
\Rightarrow \quad
\begin{aligned}
x_n &= \frac{y_n}{u_{nn}}, \\
x_{n-1} &= \frac{y_{n-1} - u_{n-1,n} x_n}{u_{n-1,n-1}}, \\
x_{n-2} &= \frac{y_{n-2} - u_{n-2,n-1} x_{n-1} - u_{n-2,n} x_n}{u_{n-2,n-2}}, \\
&\ \vdots \\
x_1 &= \frac{y_1 - \sum_{j=2}^{n} u_{1j} x_j}{u_{11}}.
\end{aligned}
$$


* This is just one example of an algorithm that can be extremely efficient on the GPU, but wouldn't take advantage of the massive parallelism of the GPU
* Because translating algorithms from CPUs directly to GPUs is often inefficient, there has been a lot of emphasis on developing efficient algorithms for linear algebra on GPUs in recent years
* There are challenges both in the algorithmic / mathematical approaches for tackling linear algebra problems on the GPU as well as engineering problems
* Mathematical:
  * What algorithms are best suited for GPU parallelism?
* Engineering:
  * How do we implement an algorithm to utilize GPU resource efficiently?

## Iterative Methods
* While far from a state of the art approach, in the Navier Stokes smoke demo, we saw a different algorithm for solving a linear system that was well adapted to a GPU
* This was an iterative method called the **Jacobi method**
* In particular, when solving the Navier Stokes equations using the approach outlined [here](https://pages.cs.wisc.edu/~chaol/data/cs777/stam-stable_fluids.pdf), we need to compute a divergence free velocity field
* This turns out to involve solving a Poisson equation of the form
$$
\nabla \cdot w = \nabla^2 q
$$
* Solving this equation via finite elements or finite differences involves discretizing these equations to form a linear system of equations of the form $Ax=b$ where $A$ is given by:

$$
A =
\begin{bmatrix}
T & I &   &        &        \\
I & T & I &        &        \\
  & I & T & \ddots &        \\
  &   & \ddots & \ddots & I \\
  &   &        & I & T
\end{bmatrix},
\qquad
T =
\begin{bmatrix}
-4 & 1  &        &        &        \\
 1 & -4 & 1      &        &        \\
   &  1 & -4     & 1      &        \\
   &    & \ddots & \ddots & \ddots \\
   &    &        & 1      & -4
\end{bmatrix},
\qquad
I =
\begin{bmatrix}
1 &        &        &        \\
  & 1      &        &        \\
  &        & \ddots &        \\
  &        &        & 1
\end{bmatrix}.
$$

* The Jacobi method decomposes a matrix $A$ into a digonal matrix $D$, a lower triangular part $L$, and an upper triangular part $U$
$$
D = \begin{bmatrix} a_{11} & 0 & \cdots & 0 \\ 0 & a_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\0 & 0 & \cdots & a_{nn} \end{bmatrix} \text{ and } L+U = \begin{bmatrix} 0 & a_{12} & \cdots & a_{1n} \\ a_{21} & 0 & \cdots & a_{2n} \\ \vdots & \vdots & \ddots & \vdots \\ a_{n1} & a_{n2} & \cdots & 0 \end{bmatrix}$$
$$

* Using this decomposition, the solution $x$ is computed iteratively via
$$
 \mathbf{x}^{(k+1)} = D^{-1} (\mathbf{b} - (L+U) \mathbf{x}^{(k)})
$$

* The basic idea here is that we write
$$
Ax = (D + L + U)x = b
$$
then multiply by $D^{-1}$ to get
$$
(I + D^{-1} L + D^{-1} U)x = D^{-1}b  
$$.
* Simplifying gives
$$
x = D^{-1} (b - (L + U)x)
$$
* Then since we have two $x$'s here, we assume that we can use the previous iteration of an iterative method as a reasonable guess so that we get the iterative update formula shown above.

## Strengths and Weaknesses of the Jacobi Method

**Strengths**
* One really nice thing about this formula is that every step involves computations that are fairly efficient on a GPU
* In particular, both of these can be parallelized efficiently:
    * Matrix multiplication
    * Vector addition
* For instance we have vector addition, and matrix multiplication.
* Computing and multiplying by the inverse $D^{-1}$ is also trivial since $D$ is diagonal

**Weaknesses**
* The Jacobi method is not a great iterative solver
* It tends to converge to a solution slowly, requiring many iterations
* It does not work for every problem! It is best suited for diagonally dominant problems

##

## The Jacobi Method for The Poisson Equation

* In the case of the Poisson equation we can prety easily write out the Jacobi iteration step
* At each iteration the value of $u$ in the $i,j$ cell is updated as
$$
u^{(k+1)}_{i,j}
= \frac{1}{4}\left(
u^{(k)}_{i+1,j}
+ u^{(k)}_{i-1,j}
+ u^{(k)}_{i,j+1}
+ u^{(k)}_{i,j-1}
- h^2 f_{i,j}
\right)
$$
* Here $h$ is the cell of each grid
* Hence, at each iteration, the value in a cell is dependent on the values of its neighbors
* Written as a CUDA kernel in Numba, this became

```python
def jacobi_pressure(p_new, p, div, inv_dx):
    x, y = cuda.grid(2)
    if x >= W or y >= H: return
    xm = x - 1 if x > 0 else x
    xp = x + 1 if x < W-1 else x
    ym = y - 1 if y > 0 else y
    yp = y + 1 if y < H-1 else y
    p_new[y, x] = 0.25 * (p[y, xp] + p[y, xm] + p[yp, x] + p[ym, x] - div[y, x] * (DX*DX))
```

